<a href="https://colab.research.google.com/github/Mesgarin/emotion_project/blob/main/kaggle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report
from keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, BatchNormalization, MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, ReLU, Flatten
from keras.optimizers import AdamW
from keras.initializers import  RandomNormal
from keras.callbacks import ModelCheckpoint, TensorBoard
#from keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf

In [5]:
import kagglehub
import os

# Download dataset
path = kagglehub.dataset_download("msambare/fer2013")

# Inspect path
print("Dataset downloaded to:", path)
print(os.listdir(path))

# Define training/testing directories
TRAIN_DIR = os.path.join(path, "train")
TEST_DIR = os.path.join(path, "test")

print("TRAIN_DIR:", TRAIN_DIR)
print("TEST_DIR:", TEST_DIR)


Using Colab cache for faster access to the 'fer2013' dataset.
Dataset downloaded to: /kaggle/input/fer2013
['test', 'train']
TRAIN_DIR: /kaggle/input/fer2013/train
TEST_DIR: /kaggle/input/fer2013/test


In [6]:
# Constants
IMAGE_WIDTH = 48
IMAGE_HEIGHT = 48
IMAGE_DEPTH = 1
BATCH_SIZE = 64
EPOCHS = 100

In [7]:
# ---- TRAIN + VALIDATION (80/20) ----
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,  # 20% validation

    # ----- Augmentations -----
    rotation_range=10,          # xoay nhẹ ±10 độ
    width_shift_range=0.1,      # dịch ngang ±10%
    height_shift_range=0.1,     # dịch dọc ±10%
    zoom_range=0.1,             # zoom ±10%
    shear_range=0.1,            # shear nhẹ
    brightness_range=[0.8, 1.2],# tăng/giảm sáng
    horizontal_flip=True,       # lật ngang (phổ biến trên face/emotion)

    fill_mode='nearest'         # tránh tạo noise khi shift
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2,

    # ----- Rotation / shift / scale / shear -----
    rotation_range=20,         # ±20 độ, tăng so với trước
    width_shift_range=0.15,    # ±15%
    height_shift_range=0.15,   # ±15%
    zoom_range=0.15,           # ±15%
    shear_range=0.15,          # ±15%

    # ----- Brightness & contrast -----
    brightness_range=[0.6, 1.4], # ±40% ánh sáng

    # ----- Horizontal flip -----
    horizontal_flip=True,

    # ----- Fill mode -----
    fill_mode='nearest'
)

test_datagen = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

print("Loading Training Data...")
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    subset='training'
)

print("Loading Validation Data...")
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    subset='validation'
)

# ---- TEST DATA (GIỮ NGUYÊN) ----
print("Loading Test Data...")
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    color_mode='grayscale',
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

# Auto detect labels
CLASS_NAMES = list(train_generator.class_indices.keys())
NUM_CLASSES = train_generator.num_classes
print(f"Detected {NUM_CLASSES} classes: {CLASS_NAMES}")

Loading Training Data...
Found 22968 images belonging to 7 classes.
Loading Validation Data...
Found 5741 images belonging to 7 classes.
Loading Test Data...
Found 7178 images belonging to 7 classes.
Detected 7 classes: ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']


In [8]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, BatchNormalization, ReLU
from tensorflow.keras.layers import MaxPooling2D, Dropout, GlobalAveragePooling2D, Dense, Add

def residual_block(x, filters):
    shortcut = x

    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)
    x = ReLU()(x)

    x = Conv2D(filters, 3, padding='same', kernel_initializer='he_normal')(x)
    x = BatchNormalization()(x)

    x = Add()([shortcut, x])
    x = ReLU()(x)
    return x


inputs = Input(shape=(48,48,1))

# Block 1
x = Conv2D(64, 3, padding='same', kernel_initializer='he_normal')(inputs)
x = BatchNormalization()(x)
x = ReLU()(x)
x = residual_block(x, 64)
x = MaxPooling2D(2)(x)
x = Dropout(0.25)(x)

# Block 2
x = Conv2D(128, 3, padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = residual_block(x, 128)
x = MaxPooling2D(2)(x)
x = Dropout(0.25)(x)

# Block 3
x = Conv2D(256, 3, padding='same', kernel_initializer='he_normal')(x)
x = BatchNormalization()(x)
x = ReLU()(x)
x = residual_block(x, 256)
x = MaxPooling2D(2)(x)
x = Dropout(0.4)(x)

# GAP + Dense
x = GlobalAveragePooling2D()(x)
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)
outputs = Dense(7, activation='softmax')(x)

model = Model(inputs, outputs)
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 48, 48, 1) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 48, 48,    │        640 │ input_layer[0][0] │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 48, 48,    │        256 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu (ReLU)        │ (None, 48, 48,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 48, 48,    │     36,928 │ re_lu[0][0]       │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 48,    │        256 │ conv2d_1[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_1 (ReLU)      │ (None, 48, 48,    │          0 │ batch_normalizat… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 48, 48,    │     36,928 │ re_lu_1[0][0]     │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 48, 48,    │        256 │ conv2d_2[0][0]    │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 48, 48,    │          0 │ re_lu[0][0],      │
│                     │ 64)               │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_2 (ReLU)      │ (None, 48, 48,    │          0 │ add[0][0]         │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 24, 24,    │          0 │ re_lu_2[0][0]     │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 24, 24,    │          0 │ max_pooling2d[0]… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 24, 24,    │     73,856 │ dropout[0][0]     │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalizatio… │ (None, 24, 24,    │        512 │ conv2d_3[0][0]    │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ re_lu_3 (ReLU)      │ (None, 24, 24,    │          0 │ batch_normalizat… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 24, 24,    │    147,584 │ re_lu_3[0][0]   

 Total params: 1,991,815 (7.60 MB)

 Trainable params: 1,989,127 (7.59 MB)

 Non-trainable params: 2,688 (10.50 KB)

In [9]:
model.compile(loss='categorical_crossentropy', optimizer=AdamW(), metrics=['accuracy'])


In [ ]:
import pickle
# -------------------------
# CALLBACKS
# -------------------------

checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath='best_model.h5',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

# -------------------------
# TRAINING
# -------------------------
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=validation_generator,
    callbacks=[checkpoint]
)

# 🔥 SAVE HISTORY (thêm đoạn này)
import pickle
with open("train_history.pkl", "wb") as f:
    pickle.dump(history.history, f)
print("Saved training history to train_history.pkl")

# -------------------------
# LOAD BEST MODEL
# -------------------------

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/100
128/359 ━━━━━━━━━━━━━━━━━━━━ 23:37 6s/step - accuracy: 0.2082 - loss: 2.1894

In [ ]:
def visualize_train_history(history):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))

    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train')
    axes[0].plot(history.history['val_accuracy'], label='Validation')
    axes[0].set_title('Model Accuracy')
    axes[0].set_ylabel('Accuracy')
    axes[0].set_xlabel('Epoch')
    axes[0].legend()

    # Loss
    axes[1].plot(history.history['loss'], label='Train')
    axes[1].plot(history.history['val_loss'], label='Validation')
    axes[1].set_title('Model Loss')
    axes[1].set_ylabel('Loss')
    axes[1].set_xlabel('Epoch')
    axes[1].legend()

    plt.tight_layout()
    plt.show()



with open("/kaggle/working/train_history.pkl", "rb") as f:
    saved_history = pickle.load(f)

visualize_train_history(saved_history)


In [ ]:
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

# -------------------------
# LOAD BEST MODEL
# -------------------------
# Nếu bạn lưu weights với ModelCheckpoint
model.load_weights("/kaggle/input/fer-rgb/best_model_newcnn.h5")
print("Loaded best model with lowest validation loss.")

# Nếu bạn lưu cả model bằng model.save("shufflenetv2_grayscale")
# model = tf.keras.models.load_model("shufflenetv2_grayscale")

# -------------------------
# EVALUATE ON TEST SET
# -------------------------
def report_test_results():
    print("Evaluating on Test Set...")
    # Evaluate returns [loss, accuracy]
    loss, accuracy = model.evaluate(test_generator)
    print(f"Test Accuracy: {accuracy*100:.2f}%")

    # Make predictions
    print("Generating predictions...")
    predictions = model.predict(test_generator, verbose=1)

    # Convert predictions to class indexes
    y_pred_indices = np.argmax(predictions, axis=1)

    # Get true labels directly from the generator
    y_true_indices = test_generator.classes

    # Get the class names (labels)
    class_labels = list(test_generator.class_indices.keys())

    # Classification Report
    print("\nClassification Report:\n")
    print(classification_report(y_true_indices, y_pred_indices, target_names=class_labels))

    # Confusion Matrix
    cm = confusion_matrix(y_true_indices, y_pred_indices)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_labels, yticklabels=class_labels)
    plt.xlabel('Predicted Label')
    plt.ylabel('True Label')
    plt.title('Confusion Matrix')
    plt.show()

# Call the function
report_test_results()

In [ ]:
def predict_random_samples():
    # Grab a single batch of images
    # We use next() to fetch the first batch from the generator
    images, labels = next(validation_generator)

    # Pick 5 random indices from this batch (batch size is usually 32)
    indices = np.random.choice(len(images), 5, replace=False)

    # Get class names map {0: 'angry', 1: 'happy', ...}
    class_map = {v: k for k, v in validation_generator.class_indices.items()}

    fig, axes = plt.subplots(1, 5, figsize=(20, 4))

    for i, idx in enumerate(indices):
        img = images[idx]

        # Get True Label
        true_idx = np.argmax(labels[idx])
        true_label = class_map[true_idx]

        # Get Prediction
        # Add extra dim because model expects (Batch, Height, Width, Channel)
        pred_prob = model.predict(np.expand_dims(img, axis=0), verbose=0)
        pred_idx = np.argmax(pred_prob)
        pred_label = class_map[pred_idx]

        # Display Image
        # Squeeze removes the channel dim (48,48,1) -> (48,48) for plotting
        axes[i].imshow(img.squeeze(), cmap='gray')
        axes[i].axis('off')

        # Title color: Green if correct, Red if wrong
        color = 'green' if true_label == pred_label else 'red'
        axes[i].set_title(f"True: {true_label}\nPred: {pred_label}", color=color)

    plt.show()

predict_random_samples()